# 도매 가격 데이터 종합

### import

In [ ]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

### 윤년(2016년) 예외처리용 함수

In [ ]:
def check_str(item):
    if '/' in item:
        date_ls = item.split('/')
        item = '2022' + '-' + date_ls[0] + '-' + date_ls[1]
    return item

### 도매 가격 데이터 종합셋 작성 및 저장

In [ ]:
for season in ['봄', '여름', '가을', '겨울'] :
    result = pd.DataFrame()
    for year in range(13, 22):
        df_raw = pd.read_excel(f'../data/도매가/원본/20{year}_5대도시_{season}배추_도매가격정보.xlsx')
    
        df = df_raw.T
        df = df.rename(columns = df.iloc[0])
        df.drop(['구분'], inplace = True)

        df.reset_index(drop = False, inplace = True)
        df.rename(columns = {'index':'날짜'}, inplace = True)
        df = df[df != '-']

        for i in df.columns:
            df[i] = df[i].astype(str)
            if i == '날짜':

                # 윤년 예외처리                
                df[i] = df[i].apply(lambda x : str(x))
                df[i] = df[i].apply(lambda x : check_str(x))

                df[i] = df[i].str.replace('2022', f'20{year}')
                df[i] = df[i].str[0:10]

            else:
                df[i] = df[i].str.replace(',', '')
                df[i] = df[i].astype(float)

        result = pd.concat([result, df], axis = 0, ignore_index = True)
    
    result = result.assign(품명 = season)
    result.drop(['평균', '평년'], axis = 1, inplace = True)

    ## 지역_품명 별 가격 정리
    df = result[['날짜', '품명', '서울', '부산', '대구', '광주', '대전']] # 열 순서 정렬

    retDF = pd.DataFrame()
    for raw in df.index :
        for col in ['서울', '부산', '대구', '광주', '대전'] :
            newDF = pd.DataFrame(columns = ['날짜', '지역', '품명', '가격'])
            newData = {
                '날짜' : df.loc[raw]['날짜'],
                '지역' : col,
                '품명' : df.loc[raw]['품명'],
                '가격' : df.loc[raw][col]
            }
            newDF = newDF.append(newData, ignore_index = True)   
            retDF = pd.concat([retDF, newDF], ignore_index = True)
            
    retDF.to_csv(f'../data/도매가/종합/{season}_도매_13_21년도_종합.csv', encoding='utf-8-sig')

### 도매 가격 종합 데이터셋 input

In [ ]:
path = '../data/도매가/종합/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if '종합' in file]
file_list_py

### 데이터셋 concat.

In [ ]:
retDF = pd.DataFrame()
for files in file_list_py :
    tempDF = pd.read_csv(path + files, index_col = 0)
    retDF = pd.concat([retDF, tempDF])
retDF = retDF.sort_values(by = '날짜', ascending = True)
retDF

### 데이터셋 파일 저장

In [ ]:
retDF.to_csv('../data/도매가/종합/도매_가격_완성본_13_21년도.csv', encoding = 'utf-8-sig')